# ============================================================
# 🏠 COMPILATION DES BASES LOGEMENT (2015–2022)
# ============================================================


Dans cette feuile on va : 
- Importer toutes les données
- Compiler les données dans un seul fichier pour une pluriannualité 
- Formater les noms des variables et calculer d'autres variables si nécéssaire


In [1]:
import pandas as pd
import numpy as np
import glob
import os


In [2]:
# -----------------------------------------------
# Paramètres
# -----------------------------------------------

annees = [2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013]
zone = "LIBGEO"  # zone d'agrégation
zone_f = []       # filtres éventuels
dep_filtre = ["30", "34"] # exemple : Gard

In [3]:
# -----------------------------------------------
# Chargement des fichiers Excel logement
# -----------------------------------------------
def charger_fichier_logement(fichier, sheet, annee, skip=5):
    df = pd.read_excel(fichier, sheet_name=sheet, skiprows=skip)
    df["AN"] = annee
    return df

In [4]:
# Dictionnaire des fichiers et années (ajuster selon ton dossier)
fichiers = {
    2022: ("../DATA/base-cc-logement-2022.xlsx", "COM_2022"),
    2021: ("../DATA/base-cc-logement-2021.xlsx", "COM_2021"),
    2020: ("../DATA/base-cc-logement-2020.xlsx", "COM_2020"),
    2019: ("../DATA/base-cc-logement-2019.xlsx", "COM_2019"),
    2018: ("../DATA/base-cc-logement-2018.xlsx", "COM_2018"),
    2017: ("../DATA/base-cc-logement-2017.xlsx", "COM_2017"),
    2016: ("../DATA/base-cc-logement-2016.xls", "COM_2016"),
    2015: ("../DATA/base-cc-logement-2015.xls", "COM_2015"),
    2014: ("../DATA/base-cc-logement-2020.xlsx", "COM_2014"),
    2013: ("../DATA/base-cc-logement-2019.xlsx", "COM_2013"),
}



In [5]:
import os
print("Répertoire courant :", os.getcwd())


Répertoire courant : /Users/aboubacarsiddikhamadoukaka/Documents/M2/S1/open_data/OpenData/notebooks


In [6]:
bases = []
for annee, (fichier, sheet) in fichiers.items():
    df = charger_fichier_logement(fichier, sheet, annee)
    # nettoyage des préfixes Pxx_
    df.columns = df.columns.str.replace(f"P{str(annee)[-2:]}_", "", regex=False)
    bases.append(df)
    

In [13]:
# -----------------------------------------------
# Fusion de toutes les bases
# -----------------------------------------------
histo_log = pd.concat(bases, ignore_index=True)

In [15]:
# Calcul de la variable RP_LOCPRIV qui est le nombre de résidences principales en logements locatifs privés
histo_log= histo_log.assign(RP_LOCPRIV=lambda x: x['RP'] - x['RP_LOCHLMV'])

In [17]:
histo_log = histo_log.assign(
    Plog_RP = round(100 * histo_log["RP"] / histo_log["LOG"], 1),
    Plog_RS = round(100 * histo_log["RSECOCC"] / histo_log["LOG"], 1),
    Plog_VAC = round(100 * histo_log["LOGVAC"] / histo_log["LOG"], 1),
    Plog_RP_LOCHLM = round(100 * histo_log["RP_LOCHLMV"] / histo_log["LOG"], 1),
    Plog_RP_LOCPRIV = round(100 * histo_log["RP_LOCPRIV"] / histo_log["LOG"], 1),
    Plog_RP_LOC = round(100 * histo_log["RP_LOC"] / histo_log["LOG"], 1),
    Plog_RP_PROP = round(100 * histo_log["RP_PROP"] / histo_log["LOG"], 1),
    Plog_RP_GRAT = round(100 * histo_log["RP_GRAT"] / histo_log["LOG"], 1),

    Prp_RP_LOCHLM = round(100 * histo_log["RP_LOCHLMV"] / histo_log["RP"], 1),
    Prp_RP_LOCPRIV = round(100 * histo_log["RP_LOCPRIV"] / histo_log["RP"], 1),
    Prp_RP_LOC = round(100 * histo_log["RP_LOC"] / histo_log["RP"], 1),
    Prp_RP_PROP = round(100 * histo_log["RP_PROP"] / histo_log["RP"], 1),
    Prp_RP_GRAT = round(100 * histo_log["RP_GRAT"] / histo_log["RP"], 1)
)


In [18]:
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("C22_")]


In [19]:
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("RPMAISON_AC")]
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("RPAPPART_AC")]

In [20]:
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("MEN_")]

In [22]:
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("C1")]
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("C2")]

In [23]:
print("Colonnes disponibles :", histo_log.columns.tolist())

Colonnes disponibles : ['CODGEO', 'REG', 'DEP', 'LIBGEO', 'LOG', 'RP', 'RSECOCC', 'LOGVAC', 'MAISON', 'APPART', 'RP_1P', 'RP_2P', 'RP_3P', 'RP_4P', 'RP_5PP', 'NBPI_RP', 'RPMAISON', 'NBPI_RPMAISON', 'RPAPPART', 'NBPI_RPAPPART', 'RP_ACHTOT', 'RP_ACH1919', 'RP_ACH1945', 'RP_ACH1970', 'RP_ACH1990', 'RP_ACH2005', 'RP_ACH2019', 'MEN', 'PMEN', 'PMEN_ANEM0002', 'PMEN_ANEM0204', 'PMEN_ANEM0509', 'PMEN_ANEM10P', 'NBPI_RP_ANEM0002', 'NBPI_RP_ANEM0204', 'NBPI_RP_ANEM0509', 'NBPI_RP_ANEM10P', 'RP_PROP', 'RP_LOC', 'RP_LOCHLMV', 'RP_GRAT', 'NPER_RP', 'NPER_RP_PROP', 'NPER_RP_LOC', 'NPER_RP_LOCHLMV', 'NPER_RP_GRAT', 'ANEM_RP', 'ANEM_RP_PROP', 'ANEM_RP_LOC', 'ANEM_RP_LOCHLMV', 'ANEM_RP_GRAT', 'RP_CGAZV', 'RP_CFIOUL', 'RP_CELEC', 'RP_CGAZB', 'RP_CAUT', 'RP_ELEC', 'RP_EAUCH', 'RP_BDWC', 'RP_CHOS', 'RP_CLIM', 'RP_TTEGOU', 'RP_GARL', 'RP_VOIT1P', 'RP_VOIT1', 'RP_VOIT2P', 'RP_HABFOR', 'RP_CASE', 'RP_MIBOIS', 'RP_MIDUR', 'AN', 'RP_ACH19', 'RP_ACH45', 'RP_ACH70', 'RP_ACH90', 'RP_ACH05', 'RP_ACH18', 'RP_SDB', 

In [24]:
histo_log.head()


,CODGEO,REG,DEP,LIBGEO,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,...,Plog_RP_LOCHLM,Plog_RP_LOCPRIV,Plog_RP_LOC,Plog_RP_PROP,Plog_RP_GRAT,Prp_RP_LOCHLM,Prp_RP_LOCPRIV,Prp_RP_LOC,Prp_RP_PROP,Prp_RP_GRAT
0,01001,84,01,L'Abergement-Clémenciat,379.000000,354.000000,11.000000,14.000000,369.000000,9.000000,...,3.7,89.7,12.9,79.9,0.5,4.0,96.0,13.8,85.6,0.6
1,01002,84,01,L'Abergement-de-Varey,175.000000,121.000000,41.000000,13.000000,173.000000,2.000000,...,0.0,69.1,6.9,60.6,1.7,0.0,100.0,9.9,87.6,2.5
2,01004,84,01,Ambérieu-en-Bugey,7973.370708,7107.012476,125.143124,741.215108,3578.701988,4311.645230,...,18.7,70.4,47.3,40.2,1.6,21.0,79.0,53.1,45.1,1.8
3,01005,84,01,Ambérieux-en-Dombes,920.955245,814.669985,7.749967,98.535293,673.821160,243.719043,...,8.3,80.1,22.3,64.5,1.6,9.4,90.6,25.2,73.0,1.8
4,01006,84,01,Ambléon,71.000000,57.000000,8.000000,6.000000,68.000000,3.000000,...,1.4,78.9,21.1,57.7,1.4,1.8,98.2,26.3,71.9,1.8


In [25]:
# Création de RP_LOCPRIV
if "RP_LOC" in histo_log.columns and "RP_LOCHLMV" in histo_log.columns:
    histo_log["RP_LOCPRIV"] = histo_log["RP_LOC"] - histo_log["RP_LOCHLMV"]

# Sélection des colonnes utiles
cols_to_keep = [c for c in histo_log.columns if c.endswith(
    ("LOG", "RSECOCC", "LOGVAC", "RP", "RP_LOC", "RP_PROP",
     "RP_LOCPRIV", "RP_LOCHLMV", "RP_GRAT", "AN", "RPMAISON",
     "RPAPPART", "MAISON", "APPART", "P"))
     or "RP_ACH" in c
]
histo_log = histo_log[["CODGEO","LIBGEO"] + cols_to_keep]

# Suppression des colonnes inutiles
cols_to_drop = [c for c in histo_log.columns if any(
    kw in c for kw in ["NBPI_", "HSTU", "NPER_", "ANEM_", "_ANEM", "_VOIT", "PMEN"]
)]
histo_log = histo_log.drop(columns=cols_to_drop, errors="ignore")

In [26]:
print("Colonnes disponibles :", histo_log.columns.tolist())
histo_log.head()
histo_log.info()
histo_log.shape

Colonnes disponibles : ['CODGEO', 'LIBGEO', 'DEP', 'LOG', 'RP', 'RSECOCC', 'LOGVAC', 'MAISON', 'APPART', 'RP_1P', 'RP_2P', 'RP_3P', 'RP_4P', 'RP_5PP', 'RPMAISON', 'RPAPPART', 'RP_ACHTOT', 'RP_ACH1919', 'RP_ACH1945', 'RP_ACH1970', 'RP_ACH1990', 'RP_ACH2005', 'RP_ACH2019', 'RP_PROP', 'RP_LOC', 'RP_LOCHLMV', 'RP_GRAT', 'AN', 'RP_ACH19', 'RP_ACH45', 'RP_ACH70', 'RP_ACH90', 'RP_ACH05', 'RP_ACH18', 'RP_ACH17', 'RP_ACH15', 'RP_ACH14', 'RP_ACH13', 'RP_ACH12', 'RP_ACH11', 'RP_ACH10', 'RP_LOCPRIV', 'Plog_RP', 'Plog_RP_LOCPRIV', 'Plog_RP_LOC', 'Plog_RP_PROP', 'Plog_RP_GRAT', 'Prp_RP_LOCPRIV', 'Prp_RP_LOC', 'Prp_RP_PROP', 'Prp_RP_GRAT']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349759 entries, 0 to 349758
Data columns (total 51 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CODGEO           349759 non-null  object 
 1   LIBGEO           349759 non-null  object 
 2   DEP              349759 non-null  object 
 3   LOG             

(349759, 51)

In [27]:
histo_log_filtre = histo_log = histo_log[histo_log["DEP"].astype(str).isin(dep_filtre)]

In [29]:
histo_log_filtre.head()


,CODGEO,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_LOCPRIV,Plog_RP,Plog_RP_LOCPRIV,Plog_RP_LOC,Plog_RP_PROP,Plog_RP_GRAT,Prp_RP_LOCPRIV,Prp_RP_LOC,Prp_RP_PROP,Prp_RP_GRAT
10873,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,30.213457,65.5,65.5,8.6,52.8,4.2,100.0,13.1,80.5,6.3
10874,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,53.421945,79.1,79.1,12.9,64.7,1.5,100.0,16.3,81.8,1.9
10875,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,1032.117277,74.1,66.7,26.6,45.0,2.5,90.0,35.9,60.8,3.3
10876,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,293.631063,87.1,84.3,20.8,64.4,1.9,96.8,23.9,73.9,2.1
10877,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,25.116791,43.8,43.8,10.0,31.5,2.4,100.0,22.8,71.8,5.4


In [30]:
histo_log_filtre.shape

(6931, 51)

In [31]:
# faire un table pour vois les modalités de la variable LIBGEO et le nombre de RP associé
table_libgeo = histo_log_filtre.groupby("LIBGEO")["RP"].sum().reset_index().sort_values(by="RP", ascending=False)
table_libgeo.head(20)

,LIBGEO,RP
362,Montpellier,1.505595e+06
388,Nîmes,7.124210e+05
91,Béziers,3.633009e+05
630,Sète,2.304333e+05
15,Alès,2.155292e+05
2,Agde,1.427340e+05
308,Lunel,1.105164e+05
204,Frontignan,1.029049e+05
114,Castelnau-le-Lez,1.013174e+05
45,Bagnols-sur-Cèze,8.506196e+04


In [32]:
# Exporter le fichier final dans un fichier CSV
histo_log_filtre.to_csv("../SORTIE/Compil_clean.csv", index=False)

,insee_com,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791


,insee_com,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,34340,Villetelle,34,742.852859,681.218930,26.042505,35.591424,660.878825,76.405423,5.742286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146.739481
687,34341,Villeveyrac,34,1803.742847,1584.539764,79.514844,139.688239,1549.851329,250.958565,4.823179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378.706780
688,34342,Viols-en-Laval,34,106.725770,91.223301,9.301481,6.200988,79.568954,26.123318,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.922330
689,34343,Viols-le-Fort,34,608.641341,557.720784,9.792415,41.128142,504.717578,72.249401,5.947530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.668044
